# Isuzu: Termination Classification & Maintenance Fund Projection
**Author:** Auto-generated notebook

**Purpose:**
- Apply business rules to classify vehicles (Active / Grace / Terminated)
- Estimate termination date (where applicable)
- Predict current maintenance fund and project it at termination

**How to use:**
1. Update `DATA_PATH` if your file path differs.
2. Run cells top-to-bottom.
3. Outputs: classification counts and `active_fund_predictions.csv` with predictions.


In [ ]:

# --- Imports
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import RidgeCV, LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

# Optional: plotting (matplotlib only, single-plot rule, no explicit colors)
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 160)

# --- Paths
DATA_PATH = "/mnt/data/Isuzu Standard Plans 19 Feb 2024.xlsx"  # <-- change if needed
SHEET_NAME = "FullData"
OUTPUT_PREDICTIONS_CSV = "/mnt/data/active_fund_predictions.csv"


In [ ]:

# --- Load Excel
assert os.path.exists(DATA_PATH), f"Data file not found at {DATA_PATH}"
xls = pd.ExcelFile(DATA_PATH)
print("Sheets:", xls.sheet_names)
df = pd.read_excel(DATA_PATH, sheet_name=SHEET_NAME)
print("Shape:", df.shape)
display(df.head())
print("\nColumns:", df.columns.tolist())


## Data parsing & derived fields

In [ ]:

work = df.copy()

# Numeric parsing
num_cols = ["Latest ODOMETER READING","LAST CONTRACT KM","LAST CONTRACT MONTHS",
            "MONTHS LEFT","ADMIN FEE AMOUNT","SERVICE PROVISION","INTEREST AMOUNT",
            "ACTUAL SERVICE AMOUNT","MAINTENANCE FUND"]
for c in num_cols:
    if c in work.columns:
        work[c] = pd.to_numeric(work[c], errors="coerce")

# Date parsing
date_cols = ["CONTRACT START DATE","CONTRACT END DATE","Latest ODOMETER DATE","ACTUAL TERMINATION DATE","INPUT DATE","INSTALLATION DATE"]
for c in date_cols:
    if c in work.columns:
        work[c] = pd.to_datetime(work[c], errors="coerce")

# Normalize status
if "CURRENT VEHICLE STATUS" in work.columns:
    work["CURRENT VEHICLE STATUS"] = work["CURRENT VEHICLE STATUS"].astype(str).str.upper()

# Months elapsed: prefer dates, fallback to (contract months - months left)
months_elapsed_date = ((work["Latest ODOMETER DATE"] - work["CONTRACT START DATE"]).dt.days / 30.44).round()
months_elapsed_fallback = work["LAST CONTRACT MONTHS"] - work["MONTHS LEFT"]

work["MONTHS ELAPSED"] = np.where(months_elapsed_date.notna(), months_elapsed_date, months_elapsed_fallback)

# Basic sanity checks
print("MONTHS ELAPSED describe:\n", work["MONTHS ELAPSED"].describe())


## Rule-based classification

In [ ]:

# Flags per business logic
work["reached_90k"] = work["Latest ODOMETER READING"] >= 90000
work["elapsed_60m"] = work["MONTHS ELAPSED"] >= 60
work["in_grace"] = (work["MONTHS ELAPSED"] > 60) & (work["MONTHS ELAPSED"] <= 62)

def classify_row(row):
    status_term = str(row.get("CURRENT VEHICLE STATUS", "")).upper() == "TERMINATED"
    # Grace (strict): between 60 and 62 months, not yet 90k, and not already flagged terminated
    if (row["MONTHS ELAPSED"] > 60) and (row["MONTHS ELAPSED"] <= 62) and (not row["reached_90k"]) and (not status_term):
        return "Grace"
    # Terminated if: status says so OR reached 90k OR elapsed >= 60 months
    if status_term or row["reached_90k"] or (row["MONTHS ELAPSED"] >= 60):
        return "Terminated"
    return "Active"

work["Rule_Classification"] = work.apply(classify_row, axis=1)
class_counts = work["Rule_Classification"].value_counts()
print(class_counts)


## Project termination date for Grace (if any)

In [ ]:

# For vehicles in Grace, use: Projected Termination = CONTRACT START DATE + 60 months
grace_df = work[work["Rule_Classification"] == "Grace"].copy()

if len(grace_df) > 0:
    grace_df["Projected Termination Date"] = grace_df["CONTRACT START DATE"] + pd.DateOffset(months=60)
    display(grace_df[[
        "MVA NUMBER","CUSTOMER TRADING NAME","LAST CONTRACT KM","LAST CONTRACT MONTHS",
        "Latest ODOMETER READING","CONTRACT START DATE","CONTRACT END DATE",
        "Projected Termination Date","MAINTENANCE FUND"
    ]].head(10))
else:
    print("No vehicles in Grace in this dataset snapshot.")


## Model the maintenance fund

In [ ]:

# Prepare data
monetary_cols = ["ADMIN FEE AMOUNT","SERVICE PROVISION","INTEREST AMOUNT","ACTUAL SERVICE AMOUNT","MAINTENANCE FUND"]
for col in monetary_cols:
    if col in work.columns:
        work[col] = pd.to_numeric(work[col], errors="coerce")

# Select features
numeric_features = ["Latest ODOMETER READING","MONTHS ELAPSED","ADMIN FEE AMOUNT","SERVICE PROVISION","INTEREST AMOUNT","ACTUAL SERVICE AMOUNT"]
categorical_features = ["TYPE OF PLAN","RANGE DESCRIPTION","MODEL DESCRIPTION"]

model_df = work.copy()

# Monetary nulls -> 0 for these driver fields (keeps model robust)
for col in ["ADMIN FEE AMOUNT","SERVICE PROVISION","INTEREST AMOUNT","ACTUAL SERVICE AMOUNT"]:
    if col in model_df.columns:
        model_df[col] = model_df[col].fillna(0.0)

# Drop rows missing critical target/features
model_df = model_df[model_df["MAINTENANCE FUND"].notna() & model_df["Latest ODOMETER READING"].notna() & model_df["MONTHS ELAPSED"].notna()]

X = model_df[numeric_features + categorical_features]
y = model_df["MAINTENANCE FUND"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocess: scale numeric, OneHot encode categorical (dense to avoid sparse pitfalls)
preprocess_dense = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore", min_frequency=50, sparse=False), categorical_features)
    ],
    sparse_threshold=0.0
)

# RidgeCV for stability
pipe = Pipeline(steps=[("prep", preprocess_dense), ("model", RidgeCV(alphas=(0.1, 1.0, 10.0, 100.0)))])
pipe.fit(X_train, y_train)

pred_val = pipe.predict(X_val)
r2_full = r2_score(y_val, pred_val)
mae_full = mean_absolute_error(y_val, pred_val)
print(f"Full model -> R2: {r2_full:.6f} | MAE: {mae_full:,.2f}")

# Simple linear relation (interpretability): fund ~ odometer + months
simple_features = ["Latest ODOMETER READING","MONTHS ELAPSED"]
X_simple = model_df[simple_features]
X_train_s, X_val_s, y_train_s, y_val_s = train_test_split(X_simple, y, test_size=0.2, random_state=42)

simp = Pipeline(steps=[("scale", StandardScaler()), ("lin", LinearRegression())])
simp.fit(X_train_s, y_train_s)
val_pred_simple = simp.predict(X_val_s)
r2_simple = r2_score(y_val_s, val_pred_simple)
mae_simple = mean_absolute_error(y_val_s, val_pred_simple)
print(f"Simple model -> R2: {r2_simple:.6f} | MAE: {mae_simple:,.2f}")

# Extract unscaled coefficients for slopes
lin_noscale = LinearRegression()
lin_noscale.fit(X_train_s, y_train_s)
beta_km = lin_noscale.coef_[0]
beta_month = lin_noscale.coef_[1]
print(f"Per-km slope: {beta_km:.6f} | Per-month slope: {beta_month:.6f}")


## Predict current & projected fund at termination for Active

In [ ]:

active = work[work["Rule_Classification"] == "Active"].copy()
# Remaining to triggers
active["km_remaining"] = (90000 - active["Latest ODOMETER READING"]).clip(lower=0)
active["months_to_60"] = (60 - active["MONTHS ELAPSED"]).clip(lower=0)

# Prepare input for the full model
active_input = active.reindex(columns=numeric_features + categorical_features)
for col in ["ADMIN FEE AMOUNT","SERVICE PROVISION","INTEREST AMOUNT","ACTUAL SERVICE AMOUNT"]:
    if col in active_input.columns:
        active_input[col] = pd.to_numeric(active_input[col], errors="coerce").fillna(0.0)

# Drop rows with missing key predictors
mask_complete = active_input["Latest ODOMETER READING"].notna() & active_input["MONTHS ELAPSED"].notna()
active_complete = active.loc[mask_complete].copy()
active_input_complete = active_input.loc[mask_complete].copy()

# Current fund prediction
active_complete["Pred_Current_Fund"] = pipe.predict(active_input_complete)

# Projected change to termination using simple slopes
delta_by_km = beta_km * active_complete["km_remaining"]
delta_by_time = beta_month * active_complete["months_to_60"]
active_complete["Pred_Fund_At_Termination"] = active_complete["Pred_Current_Fund"] + delta_by_km + delta_by_time

# Assemble report
report_cols = [
    "MVA NUMBER","CUSTOMER TRADING NAME","TYPE OF PLAN","RANGE DESCRIPTION","MODEL DESCRIPTION",
    "Latest ODOMETER READING","MONTHS ELAPSED","km_remaining","months_to_60",
    "MAINTENANCE FUND","Pred_Current_Fund","Pred_Fund_At_Termination"
]

predictions_df = active_complete[report_cols].copy()

print(f"Active vehicles total: {len(active)} | Predictable subset: {len(active_complete)}")
display(predictions_df.head(20))

# Save to CSV
predictions_df.to_csv(OUTPUT_PREDICTIONS_CSV, index=False)
print("Saved predictions to:", OUTPUT_PREDICTIONS_CSV)


## Quick summary & (optional) visualization

In [ ]:

# At-risk: predicted negative fund at termination
at_risk = predictions_df[predictions_df["Pred_Fund_At_Termination"] < 0]
surplus = predictions_df[predictions_df["Pred_Fund_At_Termination"] >= 0]

print(f"At-risk vehicles (predicted deficit at termination): {len(at_risk)}")
print(f"Surplus vehicles (predicted >= 0 at termination): {len(surplus)}")
print("Total predicted deficit (sum):", at_risk["Pred_Fund_At_Termination"].sum().round(2))
print("Total predicted surplus (sum):", surplus["Pred_Fund_At_Termination"].sum().round(2))

# Optional: simple histogram (single plot, default colors)
plt.figure()
predictions_df["Pred_Fund_At_Termination"].hist(bins=50)
plt.title("Predicted Maintenance Fund at Termination (Active vehicles)")
plt.xlabel("Predicted Fund at Termination")
plt.ylabel("Count")
plt.show()
